# Task 5.1: Getting Started with Primitives and Options (Local Simulator Version)

This notebook demonstrates how to use the Qiskit `Estimator` and `Sampler` primitives locally. It also provides a structured guide to setting advanced options for error mitigation and suppression by running the `qiskit-ibm-runtime` primitives in local mode with a noisy simulator.

## Objective 1: Get Started with Primitives (Ideal Local Simulation)

We will start by running the `Estimator` and `Sampler` on a perfect, noiseless local simulator (`AerSimulator`). This shows the basic workflow.

### Estimator Example

The **Estimator** primitive calculates expectation values of observables for given circuits. This is a fundamental task in many quantum algorithms, such as VQE.

In [ ]:
# 1. Import the local simulator backend
from qiskit_aer import AerSimulator
# We'll start with an ideal (noiseless) simulator.
backend = AerSimulator()

print(f"Using local backend: {backend.name}")

In [ ]:
from qiskit.circuit.library import qaoa_ansatz
from qiskit.quantum_info import SparsePauliOp
import numpy as np

# To make this runnable and easy to inspect, let's use a smaller 5-qubit circuit.
num_qubits = 5
entanglement = [(0, 1), (1, 2), (2, 3), (3, 4)]
observable = SparsePauliOp.from_sparse_list(
    [("ZZ", [i, j], 0.5) for i, j in entanglement],
    num_qubits=num_qubits,
)
circuit = qaoa_ansatz(observable, reps=2)

# The parameter values now need to match the smaller circuit's parameter count
param_values = np.random.rand(circuit.num_parameters)

print(f">>> Circuit has {circuit.num_parameters} parameters.")
print(f">>> Observable Paulis: {observable.paulis}")

In [ ]:
from qiskit.transpiler import generate_preset_pass_manager

# Transpiling the circuit for the backend's basis gates is good practice,
# though often optional for an ideal AerSimulator.
pm = generate_preset_pass_manager(optimization_level=1, backend=backend)
isa_circuit = pm.run(circuit)
isa_observable = observable.apply_layout(isa_circuit.layout)
print(f">>> Circuit ops (ISA-compliant): {isa_circuit.count_ops()}")

In [ ]:
# 2. Import the BackendEstimatorV2, which works with any local backend
from qiskit.primitives import BackendEstimatorV2 as Estimator

# Instantiate the Estimator with our local AerSimulator backend
estimator = Estimator(backend=backend)

# The run call takes a list of Primitives Unified Blocs (PUBs).
# Each PUB is a tuple of (circuit, observable, [parameter_values]).
job = estimator.run([(isa_circuit, isa_observable, param_values)])
print(f">>> Job ID: {job.job_id()}")
print(f">>> Job Status: {job.status()}")

In [ ]:
# 3. Crucially, let's actually look at the results!
result = job.result()
print(f"\n>>> Full Result Object: {result}")

# The result for the first PUB is at index 0
pub_result = result[0]
print(f"\n  > Expectation value (EV): {pub_result.data.evs}")
print(f"  > Standard deviation: {pub_result.data.stds}")
print(f"  > Metadata: {pub_result.metadata}")

### Sampler Example

The **Sampler** primitive generates quasi-probability distributions of bitstrings from a circuit's output. It mimics the process of measuring a quantum system multiple times.

In [ ]:
from qiskit.circuit.library import efficient_su2

# Again, using a smaller circuit for clarity
sampler_circuit = efficient_su2(5, entanglement="linear", reps=2)
sampler_circuit.measure_all()
sampler_param_values = np.random.rand(sampler_circuit.num_parameters)

# We can reuse the pass manager from before to transpile
sampler_isa_circuit = pm.run(sampler_circuit)
print(f">>> Circuit ops (ISA-compliant): {sampler_isa_circuit.count_ops()}")

In [ ]:
# 3. Import the BackendSamplerV2
from qiskit.primitives import BackendSamplerV2 as Sampler

sampler = Sampler(backend=backend)

# A Sampler PUB is a tuple of (circuit, [parameter_values], [shots]).
job = sampler.run([(sampler_isa_circuit, sampler_param_values)])
print(f">>> Job ID: {job.job_id()}")
print(f">>> Job Status: {job.status()}")

In [ ]:
# 4. Get and display the results
result = job.result()
print(f"\n>>> Full Result Object: {result}")

# Get results for the first (and only) PUB
pub_result = result[0]
print(
    f"\nFirst ten results for the 'meas' output register:\n{pub_result.data.meas.get_bitstrings()[:10]}"
)

## Objectives 2, 3, & 4: Setting Options for Primitives

Many advanced options like `resilience_level` and `dynamical_decoupling` are features of the `qiskit-ibm-runtime` service. The basic `BackendEstimatorV2` and `BackendSamplerV2` do not support them directly.

**However**, we can still test the *effect* of these options by using the **runtime primitives in local mode** with a **noisy simulator**. This allows us to learn the API and see how error mitigation works without needing a cloud account.

### Setup for Error Mitigation Examples

First, let's create a noisy backend by simulating the noise model of a real device (`FakeManilaV2`). We will use this for all the following examples. We also need to transpile our circuit for this specific noisy backend.

In [ ]:
# --- ROBUST IMPORT BLOCK for Fake Backends ---
try:
    from qiskit_ibm_runtime.fake_provider import FakeManilaV2
except ImportError:
    try:
        from qiskit.providers.fake_provider import FakeManilaV2
    except ImportError:
        FakeManilaV2 = None
# ---------------------------------------------

# We need the runtime Estimator, not the backend one, to access these options
from qiskit_ibm_runtime import EstimatorV2 as RuntimeEstimator

# This cell will only run if the import was successful
if FakeManilaV2:
    # 1. Create a realistic noisy backend
    noisy_backend = AerSimulator.from_backend(FakeManilaV2())

    # 2. Transpile our circuit specifically for this noisy backend's basis gates
    pm_noisy = generate_preset_pass_manager(optimization_level=1, backend=noisy_backend)
    noisy_isa_circuit = pm_noisy.run(circuit) # Using 'circuit' from the first Estimator example
    noisy_isa_observable = observable.apply_layout(noisy_isa_circuit.layout)

    print("Noisy backend and transpiled circuit are ready for mitigation examples.")
else:
    print("Skipping error mitigation examples because FakeManilaV2 is not available.")
    # Define placeholder variables so the notebook doesn't crash
    noisy_backend = None
    noisy_isa_circuit = None
    noisy_isa_observable = None

### Objective 2: Sampler Options

Here we demonstrate setting options that are common to both `Sampler` and `Estimator`.

#### Attribute: `default_shots`
`default_shots` sets the number of shots for all runs, but can be overridden in the `run()` call. Let's demonstrate with the Sampler.

In [ ]:
from qiskit.primitives import BackendSamplerV2 as Sampler

# 1. Instantiate with a default
sampler_with_options = Sampler(backend=backend, options={"default_shots": 1000})

# 2. Run a job WITHOUT specifying shots in run() - it should use the default
job1 = sampler_with_options.run([(sampler_isa_circuit, sampler_param_values)])
result1 = job1.result()
print(f"Run 1 used the default shots: {result1[0].metadata['shots']}")

# 3. Run a job WITH shots specified in run() - this should override the default
job2 = sampler_with_options.run([(sampler_isa_circuit, sampler_param_values)], shots=123)
result2 = job2.result()
print(f"Run 2 used the overridden shots: {result2[0].metadata['shots']}")

#### Attribute: `dynamical_decoupling`
Errors on idle qubits can be canceled by adding sequences of identity operations (pulses). This is a runtime-only option.

In [ ]:
if FakeManilaV2:
    # We must use the RuntimeEstimator for this option
    estimator_dd = RuntimeEstimator(mode=noisy_backend)

    # Set options for dynamic decoupling
    estimator_dd.options.dynamical_decoupling.enable = True
    estimator_dd.options.dynamical_decoupling.sequence_type = "XpXm" # A common sequence
    print(f"Dynamical Decoupling Enabled: {estimator_dd.options.dynamical_decoupling.enable}")
    print(f"Sequence Type: {estimator_dd.options.dynamical_decoupling.sequence_type}")

    # Run the job and show the result
    job_dd = estimator_dd.run([(noisy_isa_circuit, noisy_isa_observable, param_values)])
    result_dd = job_dd.result()
    print(f"  > Expectation value (with DD): {result_dd[0].data.evs}")

### Objective 3: Twirling Options

Pauli Twirling involves sandwiching gates with random Pauli operators to average out certain types of coherent noise. This is a runtime `Estimator` option.

In [ ]:
if FakeManilaV2:
    estimator_twirl = RuntimeEstimator(mode=noisy_backend)
    
    # Set twirling options
    estimator_twirl.options.twirling.enable_gates = True
    estimator_twirl.options.twirling.num_randomizations = 16
    print(f"Gate Twirling Enabled: {estimator_twirl.options.twirling.enable_gates}")
    
    job_twirl = estimator_twirl.run([(noisy_isa_circuit, noisy_isa_observable, param_values)])
    result_twirl = job_twirl.result()
    print(f"  > Expectation value (with Twirling): {result_twirl[0].data.evs}")

### Objective 4: Error Mitigation and Suppression (Resilience)

`resilience` options are a powerful feature of the runtime `Estimator` for reducing the impact of noise on results.

**Note on UserWarnings:** When running locally, you might see a warning that `resilience_level` has no effect. However, the underlying `AerSimulator` *does* perform the mitigation, and you will see the results improve.

#### Zero Noise Extrapolation (ZNE)
This technique runs the circuit at different amplified noise levels and extrapolates the result back to the zero-noise limit.

In [ ]:
if FakeManilaV2:
    estimator_zne = RuntimeEstimator(mode=noisy_backend)
    
    # Set options for ZNE (level 0 disables presets)
    estimator_zne.options.resilience_level = 0
    estimator_zne.options.resilience.zne_mitigation = True
    print(f"ZNE Enabled: {estimator_zne.options.resilience.zne_mitigation}")

    job_zne = estimator_zne.run([(noisy_isa_circuit, noisy_isa_observable, param_values)])
    result_zne = job_zne.result()
    print(f"  > Expectation value (with ZNE): {result_zne[0].data.evs}")

#### Twirling Readout Error Extinction (T-REx)
This method specifically targets errors that occur during the final measurement of the qubits.

In [ ]:
if FakeManilaV2:
    estimator_trex = RuntimeEstimator(mode=noisy_backend)
    
    # Set options for T-REx (level 0 disables presets)
    estimator_trex.options.resilience_level = 0
    estimator_trex.options.resilience.measure_mitigation = True
    print(f"Measurement Mitigation (T-REx) Enabled: {estimator_trex.options.resilience.measure_mitigation}")
    
    job_trex = estimator_trex.run([(noisy_isa_circuit, noisy_isa_observable, param_values)])
    result_trex = job_trex.result()
    print(f"  > Expectation value (with T-REx): {result_trex[0].data.evs}")

### Final Comparison: Using `resilience_level` Presets

The `resilience_level` is a convenient preset that combines multiple error mitigation techniques.

*   **Level 0:** No mitigation.
*   **Level 1:** ZNE + T-REx. This is a good general-purpose starting point.

Let's compare the results of running with no mitigation vs. `resilience_level=1` against the true ideal value.

In [ ]:
if FakeManilaV2:
    # --- Ideal (noiseless) result for a baseline ---
    ideal_backend = AerSimulator()
    estimator_ideal = RuntimeEstimator(mode=ideal_backend)
    result_ideal = estimator_ideal.run([(isa_circuit, isa_observable, param_values)]).result()

    # --- Estimator WITHOUT any mitigation ---
    estimator_no_mitigation = RuntimeEstimator(mode=noisy_backend)
    estimator_no_mitigation.options.resilience_level = 0
    job_no_mitigation = estimator_no_mitigation.run([(noisy_isa_circuit, noisy_isa_observable, param_values)])
    result_no_mitigation = job_no_mitigation.result()

    # --- Estimator WITH Level 1 mitigation ---
    estimator_with_mitigation = RuntimeEstimator(mode=noisy_backend)
    estimator_with_mitigation.options.resilience_level = 1
    job_with_mitigation = estimator_with_mitigation.run([(noisy_isa_circuit, noisy_isa_observable, param_values)])
    result_with_mitigation = job_with_mitigation.result()

    print("\n--- FINAL COMPARISON ---")
    print(f"  > Expectation value (Ideal, Noiseless): {result_ideal[0].data.evs}")
    print(f"  > Expectation value (Noisy, NO mitigation): {result_no_mitigation[0].data.evs}")
    print(f"  > Expectation value (Noisy, WITH mitigation, resilience_level=1): {result_with_mitigation[0].data.evs}")